# Document Splitting

In [ ]:
import os
import openai
import sys
sys.path.append('../..')

from py3810.myUtils import pickle_dump, pickle_load
path_lumen_docs = '..\langchain\docs\lumen\\docs\\'

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv('.env\my_api_key.env')) # read local .env file

SECRET_KEY1 = os.environ.get("SECRET_KEY")
DATABASE_PASSWORD2 = os.environ.get("DATABASE_PASSWORD")
print(f"SECRET_KEY = {SECRET_KEY1}")
print(f"DATABASE_PASSWORD = {DATABASE_PASSWORD2}")

openai.api_key  = os.environ['openai_api']

In [ ]:
from urllib.parse import urlparse

def url_validator(x):
    try:
        result = urlparse(x)
        # print(f'result: {result}, {type(result)}')
        return all([result.scheme, result.netloc])
    except AttributeError:
        return False

In [ ]:
def remove_duplicates(my_list):
  """
  Removes duplicate items from a list while preserving order.

  Args:
      my_list: A list with potential duplicates.

  Returns:
      A new list with duplicates removed, preserving the original order.
  """
  seen = set()
  unique_list = []
  for item in my_list:
    if item not in seen:
      seen.add(item)
      unique_list.append(item)
  return unique_list

# Example usage
my_list = [1, 2, 2, 3, 4, 1]
unique_list = remove_duplicates(my_list)
print(unique_list)  # Output: [1, 2, 3, 4] (Order may vary)

In [ ]:
from pyrsistent import v
import requests
from bs4 import BeautifulSoup
from urllib.parse import urlparse

def url_validator(x):
    try:
        result = urlparse(x)
        # print(f'result: {result}, {type(result)}')
        return all([result.scheme, result.netloc])
    except AttributeError:
        return False

def list_website_links(url, verbose=False):
  l_hrefs = []
  l_valid_hrefs = []  
  """Fetches the HTML content of a website and extracts all links."""
  response = requests.get(url)
  soup = BeautifulSoup(response.content, "html.parser")
  links = soup.find_all('a')
  
  for link in links:
    # Get the href attribute which contains the link URL
    href = link.get('href')
    # Check if the link exists and avoid empty links
    if href:
      # print(f'href: {href}')
      l_hrefs.append(href)      
      if url_validator(href):
        l_valid_hrefs.append(href)
        # print(f'valid_urls: {href}')

  if verbose:
    print(f'valid hrefs, len={len(l_valid_hrefs)}:')
    for valid_href in l_valid_hrefs:
      print(valid_href)
    print(f'\nall hrefs, len={len(l_hrefs)}:')
    for href in l_hrefs:
      print(href)       

  return l_valid_hrefs, l_hrefs       

# # Example usage
# website_url = "https://www.lumenoptometric.com"
# l_valid_hrefs, l_hrefs  = list_website_links(website_url, verbose=True)


In [ ]:
import requests
from bs4 import BeautifulSoup

def find_youtube_links(url):
  """
  This function takes a URL as input and returns a list of YouTube video links found on the webpage.

  Args:
      url: The URL of the webpage to scrape.

  Returns:
      A list of YouTube video links found on the webpage.
  """
  try:
    # Get the webpage content
    response = requests.get(url)
    response.raise_for_status()  # Raise an exception for bad status codes

    # Parse the HTML content
    soup = BeautifulSoup(response.content, 'html.parser')

    # Find all iframe tags (common for embedding videos)
    iframes = soup.find_all('iframe')

    # Extract YouTube video links from iframes
    youtube_links = []
    for iframe in iframes:
      # Check if iframe source points to YouTube
      if 'youtube.com/embed/' in iframe.get('src', ''):
        youtube_links.append(iframe['src'])

    return youtube_links

  except requests.exceptions.RequestException as e:
    print(f"Error: {e}")
    return []

# Example usage
target_url = "https://www.lumenoptometric.com/"  # Replace with the target webpage URL
youtube_links = find_youtube_links(target_url)

if youtube_links:
  print("Found YouTube links:")
  for link in youtube_links:
    print(link)
else:
  print("No YouTube links found on the webpage.")


In [ ]:
def find_substring(text_list, substring):
  """
  Searches a list of text strings for a given substring and returns a new list with matches.

  Args:
      text_list: A list of strings.
      substring: The substring to search for.

  Returns:
      A new list containing all strings from text_list that have the specified substring.
  """
  return [text for text in text_list if substring in text]  # Case sensitive search

# Example usage
text_list = ["This has abc", "another string", "no xyz here", "AbCdEf", "fdsabcieojdsl", "fdsab cieojdsl"]
matches_abc = find_substring(text_list, substring="abc")
print(matches_abc)  # Output: ["This has abc", "AbCdEf"]

matches_xyz = find_substring(text_list, substring="xyz")
print(matches_xyz)  # Output: ["no xyz here"]



In [ ]:
valid_urls = []
yt_links = []
website_url = "https://www.lumenoptometric.com"
_urls, _  = list_website_links(website_url, verbose=False)
valid_urls = valid_urls + _urls
valid_urls = remove_duplicates(valid_urls)
print(f'valid_urls, len:{len(valid_urls)}:\n{valid_urls}')

_yt_links = find_youtube_links(_urls)
yt_links = yt_links + _yt_links
yt_links = remove_duplicates(yt_links)
print(f'yt_links, len:{len(yt_links)}:\n{yt_links}')

In [ ]:
# website_url = "https://www.lumenoptometric.com/blog/"
# _urls, _  = list_website_links(website_url, verbose=False)

In [ ]:
for _url in _urls:
  _urls = find_substring(_urls, 'https://www.lumenoptometric.com')  
  _tmp_urls, _ = list_website_links(_url, verbose=False)
  valid_urls = valid_urls + _tmp_urls
  valid_urls = find_substring(valid_urls, 'https://www.lumenoptometric.com')    
  valid_urls = remove_duplicates(valid_urls)

  _yt_links = find_youtube_links(_url)
  yt_links = yt_links + _yt_links
  yt_links = remove_duplicates(yt_links)    

print(f'valid_urls, len:{len(valid_urls)}:\n{valid_urls}')
print(f'yt_links, len:{len(yt_links)}:\n{yt_links}')
_urls = _tmp_urls

# print(f'_tmp_urls, len:{len(_tmp_urls)}:\n{_tmp_urls}')

# _valid_urls  = [item for sublist in _valid_urls for item in sublist]  # flatten
# _valid_urls

In [ ]:
sorted_yt_links = sorted(yt_links)
sorted_yt_links 

In [ ]:
my_list = sorted_yt_links
target_char ='?'
modified_list = [item.split(target_char)[0] for item in my_list]
print(f'len(modified_list): {len(modified_list)}')
modified_list 

In [ ]:
modified_list = remove_duplicates(modified_list)
print(f'len(modified_list): {len(modified_list)}')
modified_list 

In [ ]:
for _url in _urls:
  _tmp_urls, _ = list_website_links(_url, verbose=False)
  valid_urls = valid_urls + _tmp_urls
  valid_urls = remove_duplicates(valid_urls)

print(f'valid_urls, len:{len(valid_urls)}:\n{valid_urls}')
_urls = _tmp_urls

# print(f'_tmp_urls, len:{len(_tmp_urls)}:\n{_tmp_urls}')

# _valid_urls  = [item for sublist in _valid_urls for item in sublist]  # flatten
# _valid_urls

In [ ]:
sorted_valid_urls = sorted(valid_urls)
print(f'len(sorted_valid_urls): {len(sorted_valid_urls)}')
sorted_valid_urls

In [ ]:
lumen_urls = find_substring(sorted_valid_urls, 'https://www.lumenoptometric.com')
print(f'lumen_urls, len={len(lumen_urls)}')
lumen_urls

In [ ]:
scraped_lumen_urls = pickle_load(filename_pickle='scraped_lumen_urls', path_pickle_dump=path_lumen_docs)
print(f'len(scraped_lumen_urls): {len(scraped_lumen_urls)}')
scraped_lumen_urls = find_substring(scraped_lumen_urls, 'https://www.lumenoptometric.com')
print(f'len(scraped_lumen_urls): {len(scraped_lumen_urls)}')

In [ ]:
url_dif = list(set(scraped_lumen_urls) - set(lumen_urls))
sorted_url_dif = sorted(url_dif)
print(f'len(sorted_url_dif): {len(sorted_url_dif)}')
sorted_url_dif

In [ ]:
_urls = lumen_urls
for _url in _urls:
  _tmp_urls, _ = list_website_links(_url, verbose=False)
  lumen_urls = lumen_urls + _tmp_urls
  lumen_urls = remove_duplicates(lumen_urls)

print(f'lumen_urls, len={len(lumen_urls)}')
lumen_urls
# print(f'valid_urls, len:{len(valid_urls)}:\n{valid_urls}')
# _urls = _tmp_urls

In [ ]:
_lumen_urls = find_substring(lumen_urls, 'https://www.lumenoptometric.com')
_lumen_urls = remove_duplicates(_lumen_urls)
print(f'_lumen_urls, len={len(_lumen_urls)}')
_lumen_urls

In [ ]:
pickle_dump(file_to_pickle=_lumen_urls, filename_pickle='scraped_lumen_urls', path_pickle_dump=path_lumen_docs)

In [ ]:
_urls = lumen_urls
for _url in _urls:
  _tmp_urls, _ = list_website_links(_url, verbose=False)
  lumen_urls = lumen_urls + _tmp_urls
  lumen_urls = remove_duplicates(lumen_urls)

print(f'lumen_urls, len={len(lumen_urls)}')
lumen_urls
# print(f'valid_urls, len:{len(valid_urls)}:\n{valid_urls}')
# _urls = _tmp_urls

In [ ]:
print(f'valid_urls, len:{len(valid_urls)}:\n{valid_urls}')

In [ ]:
_valid_urls  = [item for sublist in _valid_urls for item in sublist]  # flatten
_valid_urls

In [ ]:
flatten_valid_hrefs = [item for sublist in valid_hrefs for item in sublist]
flatten_valid_hrefs 

In [ ]:
len(flatten_valid_hrefs)

In [ ]:
flatten_valid_hrefs = flatten_valid_hrefs.append(l_valid_hrefs_0)

In [ ]:
len(flatten_valid_hrefs)

flatten_valide_hrefs = flatten_valide_hrefs.append(l_valid_hrefs_0)

In [ ]:
unique_valid_hrefs = list(set(flatten_valid_hrefs))
len(unique_valid_hrefs)
unique_valid_hrefs

In [ ]:
unique_valid_hrefs.sort()
unique_valid_hrefs

In [ ]:
for _href in l_valid_hrefs:
  l_valid_hrefs1, l_hrefs1  = list_website_links(_href, verbose=True)

In [ ]:
# Example usage
website_url = "https://www.lumenoptometric.com"
l_valid_hrefs, l_hrefs  = list_website_links(website_url)

In [ ]:
l_hrefs

In [ ]:
l_valid_hrefs

In [ ]:
import requests
from bs4 import BeautifulSoup

def list_website_links(url):
  l_links = []
  """Fetches the HTML content of a website and extracts all links."""
  response = requests.get(url)
  soup = BeautifulSoup(response.content, "html.parser")
  links = soup.find_all('a')
  
  for link in links:
    # Get the href attribute which contains the link URL
    href = link.get('href')
    # Check if the link exists and avoid empty links
    if href:
      print(href)
      l_links.append(href)
      
  return l_links      

# # Example usage
# website_url = "https://www.lumenoptometric.com/"
# result_links = list_website_links(website_url)


In [ ]:
# Example usage
website_url = "https://www.lumenoptometric.com"
l_links = list_website_links(website_url)

In [ ]:
from urllib.parse import urlparse

def url_validator(x):
    try:
        result = urlparse(x)
        # print(f'result: {result}, {type(result)}')
        return all([result.scheme, result.netloc])
    except AttributeError:
        return False

In [ ]:
valid_urls = []
for i, url in enumerate(l_links):
  # print(f'{i}, url')
  if url_validator(url):
    valid_urls.append(url)
  # else:
  #   print(f'not a valid url: {url}')  

In [ ]:
valid_urls

In [ ]:
# Example usage
website_url = "https://www.lumenoptometric.com/treehouse-eyes-myopia-management-for-children/what-is-myopia/"
list_website_links(website_url)

In [ ]:
website_url = "https://www.lumenoptometric.com/blog/"
list_website_links(website_url)